# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Cluster and Ingress

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Istio Ingress](seldon_core_setup.ipynb#Istio). Instructions [also online](./seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Configure Istio

For this example we will create the default istio gateway for seldon which needs to be called `seldon-gateway`. You can supply your own gateway by adding to your SeldonDeployments resources the annotation `seldon.io/istio-gateway` with values the name of your istio gateway.

Create a gateway for our istio-ingress

In [ ]:
!kubectl create -f resources/seldon-gateway.yaml -n istio-system

Ensure the istio ingress gatewaty is port-forwarded to localhost:8004

 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8004:80`

In [ ]:
ISTIO_GATEWAY="localhost:8004"

## Start Seldon Core

Use the setup notebook to [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core) with Istio Ingress. Instructions [also online](./seldon_core_setup.html).

## Serve Single Model

In [ ]:
!helm install mymodel ../helm-charts/seldon-single-model

In [ ]:
!helm template ../helm-charts/seldon-single-model | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-default-0-classifier

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
assert(r.success==True)
print(r)

In [ ]:
!helm delete mymodel

## Serve AB Test

In [ ]:
!helm install myabtest ../helm-charts/seldon-abtest

In [ ]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/myabtest-default-0-classifier-1
!kubectl rollout status deploy/myabtest-default-1-classifier-2

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
assert(r.success==True)
print(r)

In [ ]:
!helm delete myabtest

## Serve Multi-Armed Bandit

In [ ]:
!helm install mymab ../helm-charts/seldon-mab

In [ ]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymab-default-0-classifier-1
!kubectl rollout status deploy/mymab-default-1-classifier-2
!kubectl rollout status deploy/mymab-default-2-eg-router

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
assert(r.success==True)
print(r)

In [ ]:
!helm delete mymab

## Serve with Shadow

#### We'll use a pre-packaged model server but the 'shadow' flag can be set on any predictor.

In [ ]:
!pygmentize ./resources/istio_shadow.yaml

In [ ]:
!kubectl apply -f ./resources/istio_shadow.yaml

In [ ]:
!kubectl rollout status deploy/iris-default-0-iris-default
!kubectl rollout status deploy/iris-shadow-0-iris-shadow

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="iris",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

In [ ]:
r = sc.predict(gateway="istio",transport="rest",shape=(1,4))
assert(r.success==True)
print(r)

#### The traffic should go to both the default predictor and the shadow. If desired this can be checked in istio dashboards in the same way as with the istio canary example. When shadowing only the responses from the default predictor are used.

In [ ]:
!kubectl delete -f ./resources/istio_shadow.yaml